# Чтение данных

In [1]:
import json
from src.models import Request # Датакласс для входных данных
from src.models import Coordinates
from src.antcolony import AntAlgorithm
import src.visualisation as visualisation
moscow = Coordinates(55.75222, 37.61556)
ekaterenburg = Coordinates(56.838008, 60.597452)

In [2]:
r = Request.from_txt("data/input1.txt")
r.move_to(ekaterenburg)

print(f'Capacity: {r.capacity}\nTime limit: {r.time}\nTotal points: {r.points_number}')

Capacity: 50
Time limit: 29591
Total points: 656


In [29]:
ant_algo = AntAlgorithm(
    r, n_ants=100, n_iterations=10, alpha=1, beta=2, evaporation_rate=0.4
)

In [30]:
route = ant_algo.run()
route

[0,
 458,
 288,
 270,
 638,
 388,
 555,
 571,
 357,
 266,
 130,
 282,
 272,
 449,
 609,
 174,
 55,
 14,
 91,
 0]

In [31]:
r.cost(route)

29149.0

In [32]:
visualisation.GeoMap(r, route=route)